In [15]:
from pandas.api.types import CategoricalDtype
import pandas as pd
from scipy import stats

In [16]:
df = pd.read_csv("../../data/processed/sleep_cycle_productivity_cleaned.csv")

In [17]:
# 연속형 -> 순서형 
ordered_score = CategoricalDtype(categories=list(range(1, 11)), ordered=True)

# 적용
df['Sleep Quality'] = df['Sleep Quality'].astype(ordered_score)
df['Productivity Score'] = df['Productivity Score'].astype(ordered_score)
df['Mood Score'] = df['Mood Score'].astype(ordered_score)
df['Stress Level'] = df['Stress Level'].astype(ordered_score)

#gender 범주형 변환 
df['Gender'] = df['Gender'].astype('category')
df['Weekday'] = df['Weekday'].astype('category')

In [18]:
df.head(5)

,Date,Age,Gender,Sleep Start Time,Sleep End Time,Total Sleep Hours,Sleep Quality,Exercise (mins/day),Screen Time Before Bed (mins),Work Hours (hrs/day),Productivity Score,Mood Score,Stress Level,Weekday
0,2024-04-12,30,2,23.33,4.61,5.28,3,86,116,8.808920,8,3,6,Friday
1,2024-11-04,40,0,21.02,2.43,5.41,5,32,88,6.329833,10,3,7,Monday
2,2024-08-31,20,1,22.10,3.45,5.35,7,17,59,8.506306,10,9,10,Saturday
3,2024-02-22,30,2,23.10,6.65,7.55,8,46,80,6.070240,8,4,2,Thursday
4,2024-02-23,40,2,21.42,4.17,6.75,10,61,94,11.374994,8,7,9,Friday


In [19]:
df.shape

(5000, 14)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   Date                           5000 non-null   object  
 1   Age                            5000 non-null   int64   
 2   Gender                         5000 non-null   category
 3   Sleep Start Time               5000 non-null   float64 
 4   Sleep End Time                 5000 non-null   float64 
 5   Total Sleep Hours              5000 non-null   float64 
 6   Sleep Quality                  5000 non-null   category
 7   Exercise (mins/day)            5000 non-null   int64   
 8   Screen Time Before Bed (mins)  5000 non-null   int64   
 9   Work Hours (hrs/day)           5000 non-null   float64 
 10  Productivity Score             5000 non-null   category
 11  Mood Score                     5000 non-null   category
 12  Stress Level                   500

In [21]:
# 종속변수 이진분류: Sleep Quality 5 이상이면 1, 아니면 0
df['Sleep_Quality_Binary'] = (df['Sleep Quality'] >= 5).astype(int)
y = df['Sleep_Quality_Binary']

X = df[['Age', 'Gender', 'Sleep Start Time', 'Sleep End Time',
        'Total Sleep Hours', 'Exercise (mins/day)',
        'Screen Time Before Bed (mins)', 'Work Hours (hrs/day)',
        'Productivity Score', 'Mood Score', 'Stress Level', 'Weekday']]

In [22]:
df['Sleep Quality'].value_counts().sort_index()

Sleep Quality
1     480
2     503
3     490
4     508
5     521
6     489
7     517
8     491
9     480
10    521
Name: count, dtype: int64

In [23]:
y.value_counts().sort_index()

Sleep_Quality_Binary
0    1981
1    3019
Name: count, dtype: int64

In [24]:
df.describe()

,Age,Sleep Start Time,Sleep End Time,Total Sleep Hours,Exercise (mins/day),Screen Time Before Bed (mins),Work Hours (hrs/day),Sleep_Quality_Binary
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,33.886000,22.005350,4.980262,6.974902,43.962600,91.421200,7.987873,0.603800
std,12.320283,1.157591,1.862508,1.454033,25.798541,52.079123,2.275734,0.489156
min,10.000000,20.000000,0.560000,4.500000,0.000000,0.000000,4.000067,0.000000
25%,20.000000,21.020000,3.660000,5.690000,22.000000,46.000000,6.032720,0.000000
50%,30.000000,22.020000,4.970000,6.960000,44.000000,92.000000,7.998295,1.000000
75%,40.000000,23.000000,6.310000,8.210000,66.000000,136.000000,9.905024,1.000000
max,50.000000,23.980000,9.420000,9.500000,89.000000,179.000000,11.999141,1.000000


## 종속변수와 독립변수 간의 상관성 파악

### 반응변수 : 순서형 'Sleep Quality'

#### anova 범주형

'Age', 'Gender', 'Weekday'

In [25]:
Categorical  = df[['Age', 'Gender', 'Weekday','Productivity Score', 'Mood Score', 'Stress Level']]

In [26]:
for col in Categorical.columns:
    print("변수명",df[col].value_counts().sort_index()) 

변수명 Age
10     267
20    1164
30    1163
40    1171
50    1235
Name: count, dtype: int64
변수명 Gender
0    1675
1    1718
2    1607
Name: count, dtype: int64
변수명 Weekday
Friday       716
Monday       712
Saturday     744
Sunday       739
Thursday     689
Tuesday      720
Wednesday    680
Name: count, dtype: int64
변수명 Productivity Score
1     428
2     524
3     476
4     501
5     453
6     501
7     509
8     509
9     583
10    516
Name: count, dtype: int64
변수명 Mood Score
1     504
2     521
3     536
4     542
5     502
6     489
7     514
8     491
9     449
10    452
Name: count, dtype: int64
변수명 Stress Level
1     500
2     470
3     519
4     498
5     478
6     500
7     490
8     495
9     529
10    521
Name: count, dtype: int64


In [27]:
F_statistic, pVal = stats.f_oneway(df["Age"],y)
print("수면의 질과 Age의 상관성",F_statistic,pVal)

수면의 질과 Age의 상관성 36430.79590724647 0.0


In [28]:
F_statistic, pVal = stats.f_oneway(df["Gender"],y)
print("수면의 질과 Gender의 상관성",F_statistic,pVal)

수면의 질과 Gender의 상관성 817.2149188232029 7.696723506152129e-173


In [29]:

for col in Categorical.columns:
    categories = df[col].dropna().unique()
    groups = [y[df[col] == cat] for cat in categories]  # 각 범주에 해당하는 y 값 추출

    if len(groups) >= 2:  # 최소 2개 이상의 그룹이 있어야 ANOVA 가능
        F_statistic, pVal = stats.f_oneway(*groups)  # 그룹들을 unpack해서 넣기
        print(f"[{col}] - 수면의 질 간 ANOVA 결과: F통계량 = {F_statistic:.2f}, p = {pVal:.5f}")
        if pVal < 0.05:
            print("→ 통계적으로 유의한 차이가 있다.\n")
        else:
            print("→ 통계적으로 유의한 차이가 없다.\n")

[Age] - 수면의 질 간 ANOVA 결과: F통계량 = 0.30, p = 0.87528
→ 통계적으로 유의한 차이가 없다.

[Gender] - 수면의 질 간 ANOVA 결과: F통계량 = 0.36, p = 0.69797
→ 통계적으로 유의한 차이가 없다.

[Weekday] - 수면의 질 간 ANOVA 결과: F통계량 = 0.82, p = 0.55345
→ 통계적으로 유의한 차이가 없다.

[Productivity Score] - 수면의 질 간 ANOVA 결과: F통계량 = 0.75, p = 0.65931
→ 통계적으로 유의한 차이가 없다.

[Mood Score] - 수면의 질 간 ANOVA 결과: F통계량 = 1.31, p = 0.22476
→ 통계적으로 유의한 차이가 없다.

[Stress Level] - 수면의 질 간 ANOVA 결과: F통계량 = 1.17, p = 0.30705
→ 통계적으로 유의한 차이가 없다.



#### 스피어만 . 연속형,순서형

'Sleep Start Time', 'Sleep End Time', 'Total Sleep Hours', 'Exercise (mins/day)', 'Screen Time Before Bed (mins)', 'Work Hours (hrs/day)'

In [30]:
data_small = df[['Sleep Start Time', 'Sleep End Time',
'Total Sleep Hours', 'Exercise (mins/day)',
'Screen Time Before Bed (mins)', 'Work Hours (hrs/day)']]

In [31]:
for col in data_small.columns : 
    corr, p = stats.spearmanr(y,df[col])
    print(f" 수면의 질('Sleep Quality')과 {col} - Spearman 상관계수: {corr:.3f}, p값: {p:.4f}")
    if pVal < 0.05:
            print("→ 통계적으로 유의한 차이가 있습니다.\n")
    else:
            print("→ 통계적으로 유의한 차이가 없습니다.\n")

 수면의 질('Sleep Quality')과 Sleep Start Time - Spearman 상관계수: 0.004, p값: 0.7757
→ 통계적으로 유의한 차이가 없습니다.

 수면의 질('Sleep Quality')과 Sleep End Time - Spearman 상관계수: 0.002, p값: 0.8766
→ 통계적으로 유의한 차이가 없습니다.

 수면의 질('Sleep Quality')과 Total Sleep Hours - Spearman 상관계수: -0.002, p값: 0.8799
→ 통계적으로 유의한 차이가 없습니다.

 수면의 질('Sleep Quality')과 Exercise (mins/day) - Spearman 상관계수: 0.011, p값: 0.4293
→ 통계적으로 유의한 차이가 없습니다.

 수면의 질('Sleep Quality')과 Screen Time Before Bed (mins) - Spearman 상관계수: 0.002, p값: 0.8688
→ 통계적으로 유의한 차이가 없습니다.

 수면의 질('Sleep Quality')과 Work Hours (hrs/day) - Spearman 상관계수: -0.011, p값: 0.4247
→ 통계적으로 유의한 차이가 없습니다.

